## Filtro di Kalman

### Filtro di Kalman in uno spazio monodimensionale

https://www.kalmanfilter.net/default.aspx


Suppponiamo di essere in uno spazio bidimensionale X,Y e tracciamo la traiettoria di un veicolo.
Partiamo al tempo $t_0$ e otteniamo la prima posizione, passiamo poi alla misura $t_1$ e via dicendo proseguiamo con le altre, come nel grafico qui sotto.

![kalman](images/kalman.png)

Ci immaginiamo che l'auto stia proseguendo lungo una linea retta a velocità costante.
Dobbiamo introdurre un nuovo concetto la **curva multivariata gaussiana** questa avrà come parametri un vettore $\mu$ con le medie di ogni dimensione:

$$\mu = \begin{bmatrix}\mu_0 \\ \mu_1 \\ \vdots \\ \mu_d \end{bmatrix}$$

e una matrice $\Sigma$ che rappresenta la matrice di covarianza

$$\Sigma = \begin{bmatrix} \sigma_{11} & \sigma_{12} & \dots \\ \vdots & \ddots & \\ \sigma_{d1} &        & \sigma_{dd} \end{bmatrix} $$

Senza entrare nel dettaglio delle formule analizziamo come una curva gaussiana si presenta in uno spazio bidimensionale, i punti $x0$ e $y0$ rappresentano le medie, mentre la matrice di covarianza specifica come saranno le linee del grafico

![gaussian](images/multivariategussian.png)

Per spiegare la stima della velocità partiamo da un esempio semplice, un esempio monodimensionale.
Supponiamo di voler tracciare la posizione di un velocista sulla pista dei 100mt piani.
Come nel grafico qui sotto tracciamo la posizione del corridore nei vari istanti della corsa ovviamente all'istante t4 siamo quasi sicuri che la posizione dell'oggetto sarà 4



Ma come il filtro di kalman arriva a questa deduzione?
Il filtro crea una stima bidimensionale con una dimensione $X$ per la posizione e una dimensione $\dot X$ per la velocità, la velocità può assumere valori positivi e negativi.

All'inizio conosco la posizione ma non sappiamo la velocità, rappresentiamo questo con una curva gaussiana stretta e alta che indica il fatto che conosco bene la posizione ma non conosco la velocità
![runner](images/kalmanrunner.png)

Nel passo di predizione del punto due non posso fare molto non conoscendo la velocità, posso priettare la distribuzione su $\dot x$

![dotx](images/dotx.png)

Quello che a noi interessa in questo momento è che la posizione è correlata alla velocità, più mi muovo velocemente più arriverò lontano

Ora spostiamoci al punto 2 se disegnamo il grafico della distribuzione gaussiana ci troveremo in questa situazione.

![dotx](images/dotx2.png)

ora se moltiplichiamo la distribuzione gaussiana a priori con quella a postriori 

![dotx](images/dotx2prior.png)

otterremo una nuova distribuzione che ci da una informazione relativamente alla posizione e alla velocità

![result](images/dotx2prior_result.png)


Questo è il comportamento in linea di massima del filtro, siamo riusciti a misurare $x$ deducendolo da $\dot x$ 
Le variabili nel filtro di Kalman sono chiamate *stati*. Queste si dividono in due sottoinsiemi:

* variabili *osservabili* come la posizione in un determinato momento
* variabili *nascoste* come la velocità che non è mai direttamente osservabile

le variabili osservabili ci danno informazioni relativamente alla variabile nascosta, ad esempio una serie di misure sulla posizione ci danno una serie di informzioni relative alla velocità.

Mettiamo delle notazioni che ci aiuteranno a comprendere meglio le codifiche:

|   Simbolo  	| Significato                           	|
|:----------:	|---------------------------------------	|
|     $x$    	| posizione **prima** dello spostamento 	|
| $x \prime$ 	| posizione **dopo** lo spostamento     	|
|  $\dot x$  	| velocità nella direzione $x$          	|
| $\Delta t$ 	| durata dello spostamento "delta t"    	|


Utilizzando le terminologie della tabella sopra possiamo scrivere che:

$$x \prime = x + \dot x \Delta t$$ 

## Matrici e trasformazioni di stato

Quando si implementa il filtro di kalman dobbiamo tener conto delle seguenti variabili:

$$\begin{bmatrix} x \prime \\ \dot x \prime \end{bmatrix} \leftarrow \begin{bmatrix} a_{1} & \dots \\ \vdots & \ddots \\ \end{bmatrix} \begin{bmatrix} x \\ \dot x  \end{bmatrix}$$


se impostiamo la trasformazione per lo stato in questo modo

$$\begin{bmatrix} x \prime \\ \dot x \prime \end{bmatrix} \leftarrow \begin{bmatrix} 1 & 1 \\ 0 & 1 \end{bmatrix} \begin{bmatrix} x \\ \dot x  \end{bmatrix}$$

modo otteniamo (assumento che delta t sia unitario):

$$x \prime \leftarrow  x + \dot x$$

e 

$$\dot x \prime \leftarrow \dot x$$

mentre per le misure otteniamo la seguente:

$$z \leftarrow \begin{bmatrix} a & b \end{bmatrix} \begin{bmatrix} x \\ \dot x \end{bmatrix}$$

impostiamo il vettore in questo modo:

$$z \leftarrow \begin{bmatrix} 1 & 0 \end{bmatrix} \begin{bmatrix} x \\ \dot x \end{bmatrix}$$


## Aggiornamento

Senza entrare troppo nei dettagli non volgiamo implementare il filto ma solo introdurre il concetto impostiamo che 

$$x \prime = Fx + u$$ dove

* F rappresenta la matrice di transizione di stato
* u è il vettore di movimento


Abbiamo poi una matrice di covarianza P che calcola la nostra incertezza che 

$$P \prime = FPF^{T}$$

quello che a noi interessa capire è che in queste formule verranno utilizzate delle nuove operazioni che servono per risolvere questo problema.

## Trasformazioni lineari e matrici

Una delle cose che si dovrebbe imparare per prima nell'algebra lineare è l'idea della trasformazione lineare e il suo collegamento con le matrici.

Vediamo come si presenta una trasformazione in un sistema a due dimensioni e come questo si collega ad una moltiplicazione di matrici.

Partiamo dal concetto di trasformazione lineare, una trasformazione lineare è una funzione che "trasforma" un vettore in un altro vettore.
Esistono molti modi di "trasformare" un vettore in un altro ma noi ci occuperemo soltanto di trasformazioni "lineari".
Graficamente parlando una trasformazione lineare ha due proprietà:

* Ogni linea rimane una linea
* L'origine rimane un punto fisso

ora per creare una trasformazione su un piano possiamo creare una matrice 2x2 dove essenzialmente andiamo a codificare dove i vettori $i$ e $j$ di direzione andranno ad atterrare.

![trasformation](images/trasformation.png)

Creiamo una nuova matrice di trasformazione 

$$\begin{bmatrix} a & b \\ c & d\end{bmatrix}$$

se andiamo a moltiplicarlo con il vettore

$$\begin{bmatrix} x \\ y\end{bmatrix}$$

otteremo la seguente moltiplicazione:

$$x \begin{bmatrix} a \\ c \end{bmatrix} + y \begin{bmatrix} b \\ d \end{bmatrix} = \begin{bmatrix} ax + by \\ cx + dy \end{bmatrix}$$

Proviamo a creare delle matrici di trasformazione che possono essere utili ai nostri scopi, ad esempio creiamo una trasformazione di rotazione -90°, il vettore $i$ che prima era $\begin{bmatrix} 0 & 1 \end{bmatrix}$ ora diventa 

$$\begin{bmatrix} 0 \\ 1 \end{bmatrix}$$

Il vettore $j$ ora diventa 

$$\begin{bmatrix} -1 \\ 0 \end{bmatrix}$$

la matrice di trasformazione risultante sara 

$$\begin{bmatrix} 0 && -1 \\ 1 && 0 \end{bmatrix}$$

Un'altra trasformazione interessante chiamata "shear" fa in modo che $i$ resti fisso, perciò il vettore sarà che riguarda tale asse sarà:

$$\begin{bmatrix} 1 \\ 0 \end{bmatrix}$$

Mentre $j$ cambierà ad esempio:

$$\begin{bmatrix} 1 \\ 1 \end{bmatrix}$$

Se impostiamo una matrice con delle righe linearmente dipendenti come effetto avremo un proiezione in uno spazio di dimensione inferiore.

## Moltiplicazione di matrici come composizione

abbiamo visto come una matrice può essere usata per rappresentare una trasformazione, ora possiamo pensare di poter unire due trasformazioni assieme come ad esempio una rotazione e uno "shear".
Questa nuova trasformazione viene chiamata **composizione** possiamo rappresentare questa trasformazione anche in questo modo:

![trasformation](images/rotshear.png)

che visualmente si legge in questo modo

![trasformation](images/rotshear1.png)

L'operazione di moltiplicazioni tra matrici non è commutativa, ma è associativa.